In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import KaplanMeierFitter

import warnings
warnings.filterwarnings('ignore')

/usr/lib/python3.4/importlib/_bootstrap.py:321: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.4/importlib/_bootstrap.py:321: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('course_730.csv', engine='python')

df_org = df

df = df.drop([list(df)[0], list(df)[-3], 'final_score', 'group'], axis=1)
#df = df.drop(['hw1_score', 'hw2_score', 'hw3_score', 'student_id'], axis=1)
df = df.drop(['student_id', 'hw2_score', 'hw3_score'], axis=1)

df = df.drop(['HW1_C', 'HW1_R', 'HW1_W', 'HW1_E'], axis=1)
df = df.drop(['HW2_C', 'HW2_R', 'HW2_W', 'HW2_E'], axis=1)
df = df.drop(['HW3_C', 'HW3_R', 'HW3_W', 'HW3_E'], axis=1)

df = df.rename(index=str, columns={list(df)[-1]: "label_2",
                              list(df)[-2]: "label_1"})
df = df[df.HW1.notna()]
df = df[df.HW2.notna()]
df = df[df.HW3.notna()]
df = df[df.HW3.notna()]
df = df[df.HW2 != 'error']
df = df[df.label_2 != '?�修']
df = df[df.label_1.notna()]


In [3]:
feature_list = list(df)
hw1_df = df[feature_list[3:14]]
hw1_df.head()

,hw1_score,w1_onlineDays,w1_onlineCount,w1_onlineTime,w1_passCount,w1_failCount,w1_SyntaxError,w1_NameError,w1_IndentationError,w1_ValueError,w1_TypeError
0,25.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,25.0,2.0,1.0,6002.317,62.0,38.0,11.0,8.0,7.0,0.0,12.0
8,25.0,1.0,1.0,3470.740,14.0,18.0,14.0,0.0,2.0,0.0,0.0
9,25.0,1.0,1.0,4327.470,58.0,27.0,8.0,14.0,3.0,0.0,2.0
12,25.0,1.0,1.0,3871.781,36.0,25.0,16.0,4.0,5.0,0.0,0.0


In [ ]:
hw1_df_km = hw1_df

hw1_df_km['class'] = hw1_df_km['hw1_score'] < 24
hw1_df_km['duration'] = hw1_df_km['w1_onlineTime'] / 60
hw1_df_km['duration'] = hw1_df_km['duration'].astype(int)

hw1_df_km = hw1_df_km.drop(['hw1_score', 'w1_onlineTime'], axis=1)
hw1_df_km

# Distribution

In [ ]:
plt.figure(figsize=(15, 15))
ax = hw1_df_km.drop(['class'], axis=1).hist(alpha=0.8, bins=15, figsize=(15, 10))
#for x in ax:
#     x.set_title('x')

for array in ax:
    for subplot in array:
        subplot.set_ylabel('# of Students')
        subplot.set_xlabel('Count')

plt.tight_layout()
plt.savefig('svm_conf.png', dpi=400)

# KaplanMeierFitter

In [ ]:
kmf = KaplanMeierFitter() 
kmf.fit(durations = hw1_df_km.duration, event_observed = hw1_df_km['class'])
#kmf.event_table

In [ ]:
# plot the KM estimate
kmf.plot()
# Add title and y-axis label
plt.title("The Kaplan-Meier Estimate for Math Course")
plt.ylabel("Probability a Student Pass Homework")
plt.xlabel("Time spent on programming (minutes)")

plt.show()

# LogRank

In [ ]:
hw1_df_km['class_passCount'] = hw1_df_km['w1_passCount'] < 125
hw1_df_km = hw1_df_km.drop(['w1_passCount'], axis=1)
hw1_df_km

In [ ]:
from lifelines import KaplanMeierFitter

ix = hw1_df_km['class_passCount'] == True

plt.figure(figsize=(10, 8))
ax = plt.subplot(111)

kmf_control = KaplanMeierFitter()
ax = kmf_control.fit(hw1_df_km.loc[ix]['duration'], 
                     hw1_df_km.loc[ix]['class'], 
                     label='passCount<25').plot(ax=ax)

kmf_exp = KaplanMeierFitter()
ax = kmf_exp.fit(hw1_df_km.loc[~ix]['duration'], 
                 hw1_df_km.loc[~ix]['class'],
                 label='passCount>25').plot(ax=ax)

plt.title("The Kaplan-Meier Estimate for Math Course Homework#1")
plt.ylabel("Probability a Student Pass Homework")
plt.xlabel("Time spent on programming (minutes)")

from lifelines.plotting import add_at_risk_counts
add_at_risk_counts(kmf_exp, kmf_control, ax=ax)

plt.tight_layout()
#plt.savefig('km_hw1_passcount.png', dpi=400)